In [1]:
import torch
import numpy as np
import pickle

from HMM_Likelyhood import HMM_log_likelihood,Optimizationloop, InitializeParameters,Accumulated_HHM_LL,Optimizationloop2
import LogLikelihoodTorch

import HMM_Likelyhood

with open('.DataPhase.pickle','rb') as f:
    X = pickle.load(f) 
X_tensor = X_tensor = torch.from_numpy(X).to(torch.float64)
#X_tensor = torch.reshape(X_tensor.T,(-1,90,330))
torch.set_default_dtype(torch.float64)

def Accumulated_HHM_LL(X: Tensor,
    Pinit: Tensor,
    kappa: Tensor,
    mu: Tensor,
    Tk: Tensor,
    n: Tensor,
    p: Tensor=CONSTANTS.c0,
    K: Tensor=CONSTANTS.c1) -> Tensor:
  _0 = __torch__.HMM_Likelyhood.HMM_log_likelihood
  kappa_con = torch.minimum(^<python_value>()(kappa), torch.tensor([800]))
  _1 = torch.sqrt(torch.sum(torch.pow(mu, 2), [0]))
  mu_con = torch.div(mu, _1)
  Subjectlog_Likelihood = torch.zeros([annotate(int, n)])
  for subject in range(int(n)):
    _2 = torch.slice(X)
    _3 = torch.mul(330, subject)
    _4 = torch.mul(330, torch.add(subject, 1))
    _5 = torch.slice(_2, 1, _3, _4)
    _6 = torch.select(torch.slice(Pinit), 1, subject)
    _7 = ^<python_value>()(_6)
    _8 = ^<python_value>()(torch.select(Tk, 0, subject))
    _9 = _0(_5, _7, kappa_con, mu_con, _8, p, K, )
    _10 = torch.select(Subjectlog_Likelihood, 0, subject)
    _11 = torch.copy_(_10, _9)
  return torch.sum(Subjectlog_Likelihood)



In [12]:
def HMM_Optimization(X,kappa,mu,Tk,Pinit,learning_rate,Subjects,n_iters,K):
    

    # Initialize MixtureModel of MWD's to gain e good estimate for HMM
    pi =torch.tensor([1/K for _ in range(K)],requires_grad=True)

    Parameters_torch = [
    {'params':pi},
    {'params':kappa},
    {'params':mu},
    ]
    AdamTorch = torch.optim.Adam(Parameters_torch,lr=0.8)
    Pi_fit,Kappa_fit,mu_fit = LogLikelihoodTorch.Optimizationloop(X,[pi,kappa,mu],lose=LogLikelihoodTorch.log_likelihood,Optimizer=AdamTorch,n_iters=1500,K=K)


    Parameters_HMM= [
    {'params':Pinit},
    {'params':Kappa_fit},
    {'params':mu_fit},
    {'params':Tk},
    ]

    AdamHMM=torch.optim.Adam(Parameters_HMM,lr=learning_rate)
    AdamHMM.zero_grad()
    Pinit_fit,Kappa_fit,mu_fit,Tk_fit = Optimizationloop(X_tensor,[Pinit,Kappa_fit,mu_fit,Tk],lose=Accumulated_HHM_LL,Optimizer=AdamHMM,n=Subjects,n_iters=n_iters,K=K)



    return Pinit_fit,Kappa_fit,mu_fit,Tk_fit

In [13]:
K=5
p=90
Subjects = 199

kappa,mu,Tk,Pinit = InitializeParameters(Subjects,p,K)
Pinit_fit,Kappa_fit,mu_fit,Tk_fit = HMM_Optimization(X_tensor,kappa,mu,Tk,Pinit,learning_rate=0.4,Subjects=Subjects,n_iters=100,K=K)

epoch 1; Log-Likelihood = -5529651.945966571
epoch 101; Log-Likelihood = -6211214.377157628
epoch 201; Log-Likelihood = -6211170.850741458
epoch 301; Log-Likelihood = -6282766.537617215
epoch 401; Log-Likelihood = -6323837.7539714435
epoch 501; Log-Likelihood = -6403701.06292844
epoch 601; Log-Likelihood = -6453248.411966923
epoch 701; Log-Likelihood = -6502291.700331236
epoch 801; Log-Likelihood = -6549304.375067535
epoch 901; Log-Likelihood = -6594572.036931935
epoch 1001; Log-Likelihood = -6638415.278768824
epoch 1101; Log-Likelihood = -6681177.877256498
epoch 1201; Log-Likelihood = -6706960.088115221
epoch 1301; Log-Likelihood = -6706963.868642068
epoch 1401; Log-Likelihood = -6706968.315672187
epoch 1; Log-Likelihood = -5971647.85141992
epoch 2; Log-Likelihood = -6001492.029058575
epoch 3; Log-Likelihood = -6025625.6134384265
epoch 4; Log-Likelihood = -6038445.974589306
epoch 5; Log-Likelihood = -6043668.9926744625
epoch 6; Log-Likelihood = -6045536.6941742925
epoch 7; Log-Likelih

In [ ]:
n_iters = 500
K=5
p=90
LearningRates = [0.1,0.5,0.8,1,1.2,1.5]
Likelihoods = np.zeros((len(LearningRates),n_iters))

for idx,lr in enumerate(LearningRates):

    kappa,mu,Tk,Pinit = InitializeParameters(Subjects,p,K)
    print(lr)
    Parameters = [
        {'params':Pinit},
        {'params':kappa},
        {'params':mu},
        {'params':Tk},
    ]

    Adam = torch.optim.Adam(Parameters,lr=lr)
    Likelihoods[idx,:] = HMM_Likelyhood.OptimizationTraj(X_tensor,[Pinit,kappa,mu,Tk],lose=HMM_Likelyhood.Accumulated_HHM_LL,Optimizer=Adam,n=Subjects,n_iters=n_iters,K=K)

In [ ]:
TrajectoryData = pd.DataFrame()
#TrajectoryData['Epoch'] = np.arange(3000)
for idx,lr in enumerate(LearningRates):
    TrajectoryData[f'lr= {lr}'] = Likelihoods[idx,:]

fig=px.line(TrajectoryData,
color_discrete_sequence=px.colors.qualitative.G10
)
fig.update_layout(
    title="Learning Rate Evaluations",
    width=800,height=600,
    legend={'title':"Learning Rates"}
    #template="plotly_white",
)
fig.update_yaxes(
    title="log-Likelihood"
)
fig.update_xaxes(
    title="Epoch"
)

fig.show()

In [3]:
# Initialize parameters to tune:
K=5
p=90
Subjects = 199

kappa,mu,Tk,Pinit = InitializeParameters(Subjects,p,K)

learning_rate = 5
n_iters = 100

torch.autograd.set_detect_anomaly(False)
Parameters = [
    {'params':Pinit},
    {'params':kappa},
    {'params':mu},
    {'params':Tk},
]

Adam = torch.optim.Adam(Parameters,lr=learning_rate)



In [4]:
Pinit_fit,Kappa_fit,mu_fit,Tk_fit = Optimizationloop(X_tensor,[Pinit,kappa,mu,Tk],lose=HMM_Likelyhood.Accumulated_HHM_LL,Optimizer=Adam,n=Subjects,n_iters=200,K=K)

epoch 1; Log-Likelihood = -4801213.798400863
epoch 2; Log-Likelihood = -4824421.2699003555
epoch 3; Log-Likelihood = -4872588.986473262
epoch 4; Log-Likelihood = -4919859.683639261
epoch 5; Log-Likelihood = -4965233.35781656
epoch 6; Log-Likelihood = -5012315.667999818
epoch 7; Log-Likelihood = -5061825.692114189
epoch 8; Log-Likelihood = -5110596.680055049
epoch 9; Log-Likelihood = -5157667.049049377
epoch 10; Log-Likelihood = -5203280.828595927
epoch 11; Log-Likelihood = -5244794.642724142
epoch 12; Log-Likelihood = -5277906.3913801275
epoch 13; Log-Likelihood = -5295994.867185142
epoch 14; Log-Likelihood = -5299129.674687717
epoch 15; Log-Likelihood = -5293130.511956501
epoch 16; Log-Likelihood = -5282206.869396722
epoch 17; Log-Likelihood = -5273811.273036157
epoch 18; Log-Likelihood = -5268619.657759004
epoch 19; Log-Likelihood = -5266289.535838831
epoch 20; Log-Likelihood = -5265973.745687534
epoch 21; Log-Likelihood = -5269340.760120439
epoch 22; Log-Likelihood = -5278754.464455

In [14]:

Softmax = torch.nn.Softmax(0)
Softplus = torch.nn.Softplus()

kappa_est = Softplus(Kappa_fit)
#kappa_est = Kappa_fit
mu_est = mu /torch.sqrt((mu_fit * mu_fit).sum(axis=0))

In [15]:
print(Softmax(Tk_fit[4]))

tensor([[8.2445e-01, 1.3853e-02, 2.0000e-01, 1.9267e-04, 3.1838e-02],
        [4.2289e-02, 9.6036e-01, 2.0000e-01, 4.8228e-02, 4.3189e-02],
        [3.4627e-04, 3.1922e-04, 2.0000e-01, 1.9302e-04, 3.3402e-04],
        [1.0457e-01, 8.2372e-03, 2.0000e-01, 8.6483e-01, 1.5275e-02],
        [2.8340e-02, 1.7231e-02, 2.0000e-01, 8.6554e-02, 9.0936e-01]],
       grad_fn=<SoftmaxBackward0>)


In [16]:
import pandas as pd
labels = pd.read_csv("AALlabels.csv")
label=labels['labels'].values
import plotly.express as px
for model in range(K):
    print(f"Model {model}:  , kappa = {kappa_est[model]}")
    fig = px.bar(y=mu_est[:,model].detach().numpy(),x=label)
    #fig.update_layout(width=300)
    fig.show()

Model 0:  , kappa = 69.04614503308593


Model 1:  , kappa = 69.18382803096024


Model 2:  , kappa = 808.8360460047309


Model 3:  , kappa = 71.20058101950762


Model 4:  , kappa = 67.5627893885513


In [17]:
import pandas as pd
labels = pd.read_csv("AALlabels.csv")
label=labels['labels'].values
import plotly.express as px
for model in range(K):
    print(f"Model {model}:  , kappa = {kappa_est[model]}")
    fig = px.bar(y=mu_est[:,model].detach().numpy(),x=label)
    #fig.update_layout(width=300)
    fig.show()

Model 0:  , kappa = 69.04614503308593


Model 1:  , kappa = 69.18382803096024


Model 2:  , kappa = 808.8360460047309


Model 3:  , kappa = 71.20058101950762


Model 4:  , kappa = 67.5627893885513
